In [17]:
import pandas as pd
import json

def merge(filename:str,station_name:pd.DataFrame) -> pd.DataFrame:
    df1 = pd.read_csv(filename)
    df2 = df1.rename(columns={'trnOpDate':'日期',
                    'staCode':'車站代碼',
                    'gateInComingCnt':'進站人數',
                    'gateOutGoingCnt':'出站人數'})
    df2['日期'] = pd.to_datetime(df2['日期'].astype(str))
    result_df = pd.merge(df2,station_name,left_on='車站代碼',right_on='編碼',how='left')
    return result_df

with open('每日各站進出站人數/車站基本資料集.json',encoding='utf-8') as file:
    json_data:list[dict] = json.load(file)

stations_name = pd.DataFrame(json_data,columns=['stationCode','stationName','stationAddrTw'])
stations_name1 = stations_name.rename(columns={'stationCode':'編碼',
                      'stationName':'車站名稱',
                      'stationAddrTw':'地址'})
stations_name1['編碼'] = stations_name1['編碼'].astype(int)



In [27]:
import os
current_path = os.path.abspath(__name__) #目前執行檔案的絕對路徑
main_path = os.path.dirname(current_path) #取得目前執行的目錄的絕對路徑

'每日各站進出站人數'
dataFolder_path = os.path.join(main_path,'每日各站進出站人數') #取得放資料的目錄
filenames = os.listdir(dataFolder_path) #取得資料目錄的檔案名稱
names = list(filter(lambda name: '每日各站進出站人數' in name,filenames)) #lambda
abs_names = [os.path.join(dataFolder_path,name) for name in names] #list comprehension
all_data = [merge(filename=abs_n,station_name=stations_name1) for abs_n in abs_names] #list comprehension
result = pd.concat(all_data,ignore_index=True)
result1 = result.set_index('日期')
result2 = result1.sort_index().reindex(columns=['車站名稱','地址','進站人數','出站人數'])
result2
    



,車站名稱,地址,進站人數,出站人數
日期,,,,
2019-04-23,基隆,基隆市仁愛區港西街5號,8442,7743
2019-04-23,鎮安,屏東縣林邊鄉鎮安村永和路 4 號,15,14
2019-04-23,林邊,屏東縣林邊鄉仁和村仁愛路 33 號,368,325
2019-04-23,佳冬,屏東縣佳冬鄉六根村復興路21號,95,95
2019-04-23,東海,屏東縣枋寮鄉東海村西安路 92 號,66,50
...,...,...,...,...
2023-12-31,成功,臺中市烏日區榮泉里中山路三段 550 號,1229,3101
2023-12-31,彰化,彰化縣彰化市長樂里三民路 1 號,19617,16699
2023-12-31,花壇,彰化縣花壇鄉花壇村中正路 370 號,1864,1532
